# Level-set method
***

## Introduction
The level-set method is a method designed for modeling material interaction. The main idea is based on a function $l$, which satisfies the following.

$$ l(x) \begin{cases}
         > 0 \text{ if } x \in \Omega_1 \\
         < 0 \text{ in } x \in \Omega_2
    \end{cases}. $$

We will focus only on the interaction between two incompressible Navier-Stokes equations with different
constant parameters $\rho_i, \mu_i \in \Omega_i$ for $i  \in  \{ 1, 2 \}$. The function $l$ allows us to define
viscosity $\mu$ and density $\rho$ in the whole domain $\Omega$ as a single
function
$$ \mu(l) = \frac{1}{2}(\text{sign}(l) + 1)\mu_1 + \frac{1}{2}(\text{sign}(l) - 1) \mu_2 $$ 
and
$$ \rho(l) = \frac{1}{2}(\text{sign}(l) + 1)\rho_1 + \frac{1}{2}(\text{sign}(l) - 1) \rho_2.$$
Because we would like to solve the evolution problem, the $\Omega_i$ and the function $l$
have to depend on time. We want $l$ to be constant along streamlines for the prescribed velocity field $v$.
This condition is satisfied if
$$ \partial_t l(x, t) + \text{div}(l(x, t) v(x,  t)) = 0. $$
We can now formulate the whole system, firstly in the strong sense.
$$ \rho(l) \left( \partial_t v + (v \cdot \nabla) v \right)  = \text{div}\left(\mathbb{T}(\mu(l), \nabla v)\right) + \rho(l)g,$$

$$ \mathbb{T}(\mu(l), \nabla v) = \mu(l)\left( \nabla v + (\nabla v)^T \right) - p\mathbb{I}, $$

$$ \partial_t l + \text{div}(l v) = 0, $$

$$ \text{div}(v) = 0. $$

In this example, we will keep parameters separate from the rest of the code. This allows us to use the code easily for different problems. For that purpose, we create a Python class containing the data.

## Implementation

In [ ]:
from types import FunctionType  # This is how function in python in called. 
from dataclasses import dataclass  # Decorator
import dolfin as df


@dataclass
class Parameters():
    # Material Parametes
    mu1: float
    mu2: float
    rho1: float
    rho2: float
    eps: float
    g: df.Constant  # inner force
    # time parameters
    dt: float
    t_start: float
    t_end: float
    
    mesh: df.Mesh
    function_space: df.FunctionSpace
    bcs: list
    initial_conditions: df.Function
    sign: FunctionType


Now we start writing the main file. Firstly we create viscosity and density functions.

In [ ]:
# formulate equations
def rho(params: Parameters, l: df.Function, eps: float):
    """_summary_

    Args:
        rho1 (float): _description_
        rho2 (float): _description_
        l (df.Function): _description_
        eps (float): _description_

    Returns:
        ufl.classes.Expr: _description_
    """
    return (
        params.rho1 * 0.5* (1.0 + params.sign(l, eps))
        + params.rho2 * 0.5 * (1.0 - params.sign(l, eps))
    )


def mu(params: Parameters, l: df.Function, eps: float):
    """_summary_

    Args:
        params.mu1 (float): _description_
        mu2 (float): _description_
        l (df.Function): _description_
        eps (float): _description_

    Returns:
        ufl.classes.Expr: _description_
    """
    return (
        params.mu1 * 0.5 * (1.0 + params.sign(l, eps))
        + params.mu2 * 0.5 * (1.0 - params.sign(l, eps))
    )

Then we define functions on the function space.

In [ ]:
# Define function spaces
function_space = params.function_space
w = df.Function(function_space)  # unknown
w0 = df.Function(function_space)  # from previous step
phi = df.TestFunction(function_space)

# Split functions
v, p, l = df.split(w)
v0, p0, l0 = df.split(w0)
phi_v, phi_p, phi_l = df.split(phi)

The weak formulation of the problem can be formulated in the following way.

In [ ]:
n = df.FacetNormal(params.mesh)
h = df.CellDiameter(params.mesh)
h_avg = (h('+') + h('-'))/2.0
alpha = df.Constant(0.1)

cauchy_green = (
    2*mu(params, l, params.eps)*(df.grad(v) + df.grad(v).T)
    - p*df.Identity(params.mesh.topology().dim())
)

material_detivative = (
    (1/params.dt)*df.inner(v - v0, phi_v)  # partial time derivative
    + df.inner(df.grad(v)*v, phi_v)  # convective therm
)

momentum = (
    rho(params, l, params.eps)*material_detivative*df.dx
    + df.inner(cauchy_green, df.grad(phi_v))*df.dx
    - rho(params, l, params.eps)
    *df.inner(params.g, phi_v)*df.dx
)

mass = (
    df.div(v)*phi_p*df.dx
)

levelset_convection = (
    (1/params.dt)*df.inner(l - l0, phi_l)*df.dx
    + df.div(l*v)*phi_l*df.dx
)

stabilization = (
    alpha('+')*(h_avg**2)
    *df.inner(df.jump(df.grad(l), n), df.jump(df.grad(phi_l), n))*df.dS
)

pde_form = momentum + mass + levelset_convection + stabilization

Further, we define the Newton solver.

In [ ]:
# Set Newton-solver
J = df.derivative(pde_form, w)
ffc_options = {
    "quadrature_degree": 4,
    "optimize": True,
    "eliminate_zeros": True
}

problem = df.NonlinearVariationalProblem(pde_form, w, params.bcs, J, ffc_options)
solver = df.NonlinearVariationalSolver(problem)
prm = solver.parameters

prm['nonlinear_solver'] = 'newton'
prm['newton_solver']['linear_solver'] = 'mumps'
prm['newton_solver']['lu_solver']['report'] = False
prm['newton_solver']['absolute_tolerance'] = 1E-10
prm['newton_solver']['relative_tolerance'] = 1E-10
prm['newton_solver']['maximum_iterations'] = 20
prm['newton_solver']['report'] = True

We create the XDMF files for storing the results.

In [ ]:
# Initialize the files for writing the results
files = []
for name in ['v', 'p', 'l']:
    with df.XDMFFile(df.MPI.comm_world, f"result/{name}.xdmf") as xdmf:
        xdmf.parameters["flush_output"] = True
        files.append(xdmf)

Finally, we iterate over the time steps until the t_end is not reached.

In [ ]:
t = params.t_start
while t < params.t_end:
    df.info(f"t = {t}")
    solver.solve()
    w0.assign(w)
    t += params.dt
    # write the time-step into the file
    for func, name, xdmf in zip(w.split(True), ['v', 'p', 'l'], files):
        func.rename(name, name)
        xdmf.write(func, t)